<a href="https://colab.research.google.com/github/Slivka007/quantum-programming/blob/main/cirqsim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install cirq --quiet --no-warn-conflicts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 577.4/577.4 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.6/594.6 kB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.8/223.8 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 22.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import cirq
import numpy as np
import time

def dj_oracle(case, n):
    oracle_circuit = cirq.Circuit()
    qubits = [cirq.LineQubit(i) for i in range(n+1)]

    if case == "balanced":
        b = np.random.randint(1,2**n)
        b_str = format(b, '0'+str(n)+'b')

        for qubit in range(len(b_str)):
            if b_str[qubit] == '1':
                oracle_circuit.append(cirq.X(qubits[qubit]))

        for qubit in range(n):
            oracle_circuit.append(cirq.CNOT(qubits[qubit], qubits[n]))

        for qubit in range(len(b_str)):
            if b_str[qubit] == '1':
                oracle_circuit.append(cirq.X(qubits[qubit]))

    if case == "constant":
        output = np.random.randint(2)
        if output == 1:
            oracle_circuit.append(cirq.X(qubits[n]))

    oracle_gate = oracle_circuit.to_gate()
    oracle_gate.name = "Oracle"
    return oracle_gate

def dj_algorithm(oracle, n):
    dj_circuit = cirq.Circuit()
    qubits = [cirq.LineQubit(i) for i in range(n+1)]

    dj_circuit.append(cirq.X(qubits[n]))
    dj_circuit.append(cirq.H(qubits[n]))

    for qubit in range(n):
        dj_circuit.append(cirq.H(qubits[qubit]))

    dj_circuit.append(oracle(*qubits))

    for qubit in range(n):
        dj_circuit.append(cirq.H(qubits[qubit]))
        dj_circuit.append(cirq.measure(qubits[qubit], key=str(qubit)))

    return dj_circuit

def simulation(aer_string):
  time_sim = np.zeros(1000)
  for i in range(1000):
    n = 10
    oracle_gate = dj_oracle('balanced', n)
    dj_circuit = dj_algorithm(oracle_gate, n)

    start = time.time()

    simulator = cirq.Simulator()
    results = simulator.run(dj_circuit, repetitions=1)

    end = time.time() - start
    time_sim[i] = end

  out = "Время выполнения симуляции " + aer_string + ": " + str(np.average(time_sim))
  return out

In [ ]:
simulation('ionq.simulator')